###### December 2019 - Roger Melko

Diagonalizing the Hamiltonian matrix for the transverse field Ising model to find the energy eigenvalues and eigenkets.   Calculate the groundstate magnetization.

We will use the same Hamiltonian convention as the QMC program:
$$
H = -J\sum_{\langle i j \rangle} \sigma^z_i \sigma^z_j - B \sum_i \sigma^x_i - h \sum_i \sigma^z_i
$$
where ${\bf \sigma}_i$ are Pauli operators.  In this convention, the 1+1 CFT is at $B/J = 1$ and $h=0$.

In [2]:
using LinearAlgebra

In [13]:
N = 6
Dim = 2^N

J = 1. #exchange interaction
B = 1. #transverse field
h = 0. #longitudinal field

Hamiltonian = zeros(Dim,Dim)   #This is your 2D Hamiltonian matrix

Spin2 = 0 #give it a scope outside of the loop

for Ket = 0:Dim-1  #Loop over Hilbert Space
    Diagonal = 0.
    for SpinIndex = 0:N-2  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex)&1) - 1
        NextIndex = SpinIndex + 1
        Spin2 = 2*((Ket>>NextIndex)&1) - 1
        Diagonal = Diagonal - J*Spin1*Spin2 - h*Spin1 #spins are +1 and -1
    end
    Diagonal = Diagonal - h*Spin2 #this is the spin at the end of the chain
    
    Hamiltonian[Ket+1,Ket+1] = Diagonal
    
    for SpinIndex = 0:N-1
        bit = 2^SpinIndex   #The "label" of the bit to be flipped
        Bra = Ket ⊻ bit    #Binary XOR flips the bit
        Hamiltonian[Bra+1,Ket+1] = -B
    end
end
Hamiltonian = Hermitian(Hamiltonian);

In the Julia LinearAlgebra package, the eigen function finds eigenvalues and eigenvectors.  They are ordered; i.e. the groundstate energy corresponds to index 1

In [14]:
Diag = eigen(Hamiltonian);

In [15]:
GroundState = Diag.vectors[:, 1];  #this gives the groundstate eigenvector
Diag.values[1] / N

-1.2160383017597935

In [16]:
##### Calculate the groundstate magnetization <m^2> in the Z direction
magnetization = zeros(Dim)
abs_mag = zeros(Dim)
mag_squared = zeros(Dim)

SumSz = dropdims(sum(@. (2 * (((0:Dim-1) >> (0:N-1)') & 1) - 1); dims=2); dims=2)
AbsSumSz = abs.(SumSz)
SumSzSq = abs2.(SumSz)

magnetization = SumSz' * abs2.(Diag.vectors)
abs_mag = AbsSumSz' * abs2.(Diag.vectors)
mag_squared = SumSzSq' * abs2.(Diag.vectors)

(magnetization[1] / N), (abs_mag[1] / N), (mag_squared[1] / (N*N))

(3.0568140611345975e-14, 0.6062539621278403, 0.4824376202801941)

In [17]:
beta_vals = [20,15,10,5,3,2,1,0.8,0.5,0.2]
ED =zeros(Float64,length(beta_vals))
idx = 1
for β in beta_vals
    weights = exp.(-β * Diag.values)
    Z = sum(weights)
    E = dot(Diag.values, weights) / (N*Z)
    C = (β^2 * ((dot(Diag.values .^2, weights) / Z) - (N*E)^2))
    # magnetization of thermal state
    M = dot(weights, magnetization) / (N*Z)
    M_abs = dot(weights, abs_mag) / (N*Z)
    M2 = dot(weights, mag_squared) / (N*N*Z)
    println(β," ",E," ",C," ",M," ",M_abs," ",M2)
    #ED[idx,1] = β
    ED[idx,1] = M2
    idx += 1
end

20.0 -1.2160330882932453 0.006032382773923928 3.056486568179482e-14 0.6062678612242338 0.4824533210158867
15.0 -1.2159802506899207 0.03775820513371997 3.0531674890715147e-14 0.6064087256623791 0.48261244470733466
10.0 -1.215396031574812 0.18440870205083115 3.0164769122955446e-14 0.6079655775028978 0.4843711188876946
5.0 -1.2092191860845245 0.48255594153800985 2.63888059618621e-14 0.6236092464663624 0.5020679898280552
3.0 -1.1969216436112355 0.6373334937588169 2.0639849533464738e-14 0.6420203586364356 0.5233219205987841
2.0 -1.1747706424079871 0.9785881440130879 1.5523975962205516e-14 0.6432705416399187 0.5261570463628683
1.0 -1.0504324129922518 1.8562004466091437 7.404315618433058e-15 0.582979197736084 0.4599291425676846
0.8 -0.9709058215050207 1.9260639969018898 5.29849609973698e-15 0.5489314093266409 0.42066947056454573
0.5 -0.7523829662333139 1.5012799844898854 2.2608794817379697e-15 0.4749518770604179 0.33515869470481835
0.2 -0.3535501176598672 0.3942863030194122 3.110246134539128e